In [1]:
import pandas as pd
import numpy as np
import heartpy as hp
#import neurokit2 as nk
import emd
from scipy import signal

def truncate(x, size = 900):
    return x[size:-size]

def stat_mesaures(x):
    mean = np.mean(x)
    std = np.std(x)
    return mean,std

def hp_measures(x):
    working_data, measures = hp.process(x, 300.0,bpmmin = 40,bpmmax = 120,calc_freq=True)
    return measures.values()

In [3]:
def list_sub(x,y):
    return np.subtract(np.array(x),np.array(y))

def neuro_features(x):
    waves_raw, rpeaks = neuro_peaks(x)
    waves = pd.DataFrame.from_dict(waves_raw)
    waves.dropna(inplace=True,ignore_index=True)
    waves = waves.astype('int32')

    P_lengths = list_sub(waves['ECG_P_Offsets'],waves['ECG_P_Onsets'])
    R_lengths = list_sub(waves['ECG_R_Offsets'],waves['ECG_R_Onsets'])
    T_lengths = list_sub(waves['ECG_T_Offsets'],waves['ECG_T_Onsets'])
    w_lengths = list_sub(waves['ECG_T_Offsets'],waves['ECG_P_Onsets'])

    rel_P = np.divide(P_lengths,w_lengths)
    rel_R = np.divide(R_lengths,w_lengths)
    rel_T = np.divide(T_lengths,w_lengths)

    
    values = [x[np.array(waves['ECG_P_Peaks'])],
              x[np.array(waves['ECG_Q_Peaks'])],
              x[np.array(waves['ECG_S_Peaks'])],
              x[np.array(waves['ECG_T_Peaks'])]]
    
    
    lengths = [P_lengths,R_lengths,T_lengths,w_lengths]
    
    rels = [rel_P,rel_R,rel_T]
    
    base_features = values+lengths+rels
    
    #average features
    avg_features = []
    for f in base_features:
        avg_features.append(np.mean(f))
    
    #min features
    min_features = []
    for f in base_features:
        min_features.append(np.min(f))
        
    #max features
    max_features = []
    for f in base_features:
        max_features.append(np.max(f))
        
    return np.array(avg_features+min_features+max_features)
    


def neuro_peaks(x):
    _, rpeaks = nk.ecg_peaks(x, sampling_rate=300, show=False)
    _, waves_peak = nk.ecg_delineate(x, rpeaks, sampling_rate=300, method="dwt",show=False)
    return waves_peak,rpeaks



In [2]:
training_data = pd.read_csv('X_train.csv')
training_labels = pd.read_csv('y_train.csv')

In [3]:
training_arrays = []
for row in range(len(training_data)):
    training_arrays.append(training_data.iloc[row,1:].values[~np.isnan(training_data.iloc[row,1:])])

In [6]:
features = []
for X,y in zip(training_arrays,training_labels['y'].values):
        data = truncate(X)
        try:
            n_f = neuro_features(data)
            features.append(np.append(n_f,y))
        except:
            features.append(np.array([np.nan]*33+[y]))

C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Samuel\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Samuel\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  

In [7]:
training_data = pd.DataFrame(features)

In [11]:
hrv_features = []
for X,y in zip(training_arrays,training_labels['y'].values):
        data = truncate(X)
        mean,std = stat_mesaures(data)
        try:
            hp_m = list(hp_measures(data))
            hrv_features.append([mean,std,*hp_m,y])
        except:
            hrv_features.append([np.nan]*(len(hrv_features[0])-1)+[y])

C:\Users\Samuel\anaconda3\Lib\site-packages\scipy\interpolate\_fitpack2.py:313: UserWarning: 
A theoretically impossible result was found during the iteration
process for finding a smoothing spline with fp = s: s too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


In [18]:
w,mes = hp.process(training_arrays[0], 300.0,bpmmin = 40,bpmmax = 120,calc_freq=True)

In [19]:
mes.keys()

dict_keys(['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate', 'vlf', 'lf', 'hf', 'lf/hf', 'p_total', 'vlf_perc', 'lf_perc', 'hf_perc', 'lf_nu', 'hf_nu'])

In [21]:
measures = ['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate', 'vlf', 'lf', 'hf', 'lf/hf', 'p_total', 'vlf_perc', 'lf_perc', 'hf_perc', 'lf_nu', 'hf_nu']
training_data_hrv = pd.DataFrame(hrv_features,columns=['mean','std',*measures,'label'])

In [26]:
training_data_hrv.drop(columns = ['label'])

,mean,std,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,...,vlf,lf,hf,lf/hf,p_total,vlf_perc,lf_perc,hf_perc,lf_nu,hf_nu
0,8.820479,109.362341,72.851806,823.589744,22.080333,6.230649,9.928825,0.021277,0.000000,8.333333,...,0.0,244.167037,29.722201,8.214972,273.889238,0.0,89.148095,10.851905,89.148095,10.851905
1,2.473123,112.447296,111.168296,539.722222,203.198764,112.247701,375.275560,1.000000,1.000000,203.333333,...,0.0,0.000000,4769.343385,0.000000,4769.343385,0.0,0.000000,100.000000,0.000000,100.000000
2,8.557665,179.891864,66.271810,905.362319,34.272834,13.072518,27.542437,0.590909,0.045455,16.666667,...,0.0,523.714478,333.474090,1.570480,857.188568,0.0,61.096764,38.903236,61.096764,38.903236
3,-23.893306,209.369614,119.601329,501.666667,171.905174,179.220845,237.386230,0.700000,0.500000,45.000000,...,0.0,0.000000,8085.165105,0.000000,8085.165105,0.0,0.000000,100.000000,0.000000,100.000000
4,-19.536325,158.629010,109.756098,546.666667,120.127247,85.9371,119.706431,0.666667,0.333333,48.333333,...,0.0,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,13.466469,240.677088,75.178998,798.095238,96.162183,99.433954,115.113471,0.200000,0.200000,6.666667,...,0.0,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
5113,6.298669,77.892592,69.699903,860.833333,7.772816,4.078622,7.088127,0.000000,0.000000,5.000000,...,0.0,8.683375,28.172258,0.308224,36.855633,0.0,23.560511,76.439489,23.560511,76.439489
5114,-1.250824,173.730990,76.147088,787.948718,110.299038,66.901784,74.622507,0.333333,0.125000,75.000000,...,0.0,6053.354758,1750.888135,3.457305,7804.242894,0.0,77.564920,22.435080,77.564920,22.435080
5115,5.391590,70.402489,71.738000,836.376812,64.642555,65.976396,78.847519,0.523810,0.238095,30.000000,...,0.0,106.952816,713.598830,0.149878,820.551646,0.0,13.034258,86.965742,13.034258,86.965742


In [23]:
training_data

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,38.410714,-72.714286,-87.017857,230.125000,33.750000,41.178571,34.410714,162.285714,0.208109,0.255882,...,91.0,362.0,51.0,116.0,48.0,227.0,0.312102,0.682353,0.313333,0.0
1,34.307692,-64.461538,-177.115385,137.500000,38.000000,42.230769,39.769231,157.192308,0.243741,0.274673,...,-59.0,339.0,79.0,63.0,87.0,217.0,0.398148,0.435185,0.483333,0.0
2,100.809524,-3.666667,-296.047619,51.619048,30.428571,50.428571,43.142857,158.476190,0.196180,0.324080,...,-112.0,202.0,46.0,53.0,99.0,197.0,0.313869,0.436975,0.546961,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,18.696970,-140.545455,-527.727273,179.878788,29.757576,32.000000,42.090909,134.696970,0.220879,0.237550,...,-411.0,309.0,52.0,54.0,76.0,149.0,0.368794,0.398496,0.558824,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,259.000000,-164.692308,-223.615385,406.230769,20.384615,35.153846,31.076923,163.230769,0.127934,0.220128,...,373.0,1368.0,31.0,59.0,52.0,225.0,0.215278,0.395833,0.346667,3.0
5113,61.166667,-35.375000,-66.625000,6.500000,25.875000,36.833333,27.416667,151.750000,0.186513,0.253622,...,2.0,52.0,38.0,56.0,58.0,213.0,0.319149,0.438776,0.387097,0.0
5114,11.000000,-124.523810,-339.666667,281.476190,29.761905,41.904762,41.619048,152.000000,0.195461,0.275462,...,-221.0,624.0,44.0,56.0,90.0,182.0,0.301370,0.384615,0.604027,0.0
5115,0.636364,-48.136364,-77.954545,109.318182,25.727273,47.363636,44.545455,179.454545,0.146485,0.265306,...,-23.0,211.0,55.0,91.0,72.0,215.0,0.327381,0.486631,0.384615,0.0


In [49]:
training_all = pd.concat([training_data_hrv.drop(columns = ['label']),training_data],axis = 1)

In [57]:
training_set

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,8.820479,109.362341,72.851806,823.589744,22.080333,6.230649,9.928825,0.021277,0.000000,8.333333,...,91.000000,362.000000,51.000000,116.000000,48.000000,227.000000,0.312102,0.682353,0.313333,0.0
1,2.473123,112.447296,111.168296,539.722222,203.198764,112.247701,375.275560,1.000000,1.000000,203.333333,...,-59.000000,339.000000,79.000000,63.000000,87.000000,217.000000,0.398148,0.435185,0.483333,0.0
2,8.557665,179.891864,66.271810,905.362319,34.272834,13.072518,27.542437,0.590909,0.045455,16.666667,...,-112.000000,202.000000,46.000000,53.000000,99.000000,197.000000,0.313869,0.436975,0.546961,0.0
3,-23.893306,209.369614,119.601329,501.666667,171.905174,179.220845,237.386230,0.700000,0.500000,45.000000,...,-15.020465,425.679515,56.827737,74.462945,63.380091,207.879396,0.331707,0.485025,0.394742,1.0
4,-19.536325,158.629010,109.756098,546.666667,120.127247,85.937100,119.706431,0.666667,0.333333,48.333333,...,-411.000000,309.000000,52.000000,54.000000,76.000000,149.000000,0.368794,0.398496,0.558824,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,13.466469,240.677088,75.178998,798.095238,96.162183,99.433954,115.113471,0.200000,0.200000,6.666667,...,373.000000,1368.000000,31.000000,59.000000,52.000000,225.000000,0.215278,0.395833,0.346667,3.0
5113,6.298669,77.892592,69.699903,860.833333,7.772816,4.078622,7.088127,0.000000,0.000000,5.000000,...,2.000000,52.000000,38.000000,56.000000,58.000000,213.000000,0.319149,0.438776,0.387097,0.0
5114,-1.250824,173.730990,76.147088,787.948718,110.299038,66.901784,74.622507,0.333333,0.125000,75.000000,...,-221.000000,624.000000,44.000000,56.000000,90.000000,182.000000,0.301370,0.384615,0.604027,0.0
5115,5.391590,70.402489,71.738000,836.376812,64.642555,65.976396,78.847519,0.523810,0.238095,30.000000,...,-23.000000,211.000000,55.000000,91.000000,72.000000,215.000000,0.327381,0.486631,0.384615,0.0


In [51]:
training_all.replace([np.inf, -np.inf], np.nan, inplace=True)

In [56]:
from sklearn.impute import SimpleImputer

training_all.columns = training_all.columns.astype(str)

imp = SimpleImputer(strategy='mean')
training_set = pd.DataFrame(imp.fit_transform(training_all))

C:\Users\Samuel\anaconda3\Lib\site-packages\pandas\core\generic.py:1998: UserWarning: Warning: converting a masked element to nan.
  arr = np.asarray(values, dtype=dtype)
C:\Users\Samuel\anaconda3\Lib\site-packages\pandas\core\generic.py:1998: UserWarning: Warning: converting a masked element to nan.
  arr = np.asarray(values, dtype=dtype)


In [60]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_val, y_train, y_val = train_test_split(training_set.iloc[:,:-1].dropna(axis = 1), training_set.iloc[:,-1])

In [61]:
training_set.iloc[:,:-1]


,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,8.820479,109.362341,72.851806,823.589744,22.080333,6.230649,9.928825,0.021277,0.000000,8.333333,...,57.000000,91.000000,362.000000,51.000000,116.000000,48.000000,227.000000,0.312102,0.682353,0.313333
1,2.473123,112.447296,111.168296,539.722222,203.198764,112.247701,375.275560,1.000000,1.000000,203.333333,...,67.000000,-59.000000,339.000000,79.000000,63.000000,87.000000,217.000000,0.398148,0.435185,0.483333
2,8.557665,179.891864,66.271810,905.362319,34.272834,13.072518,27.542437,0.590909,0.045455,16.666667,...,202.000000,-112.000000,202.000000,46.000000,53.000000,99.000000,197.000000,0.313869,0.436975,0.546961
3,-23.893306,209.369614,119.601329,501.666667,171.905174,179.220845,237.386230,0.700000,0.500000,45.000000,...,53.868667,-15.020465,425.679515,56.827737,74.462945,63.380091,207.879396,0.331707,0.485025,0.394742
4,-19.536325,158.629010,109.756098,546.666667,120.127247,85.937100,119.706431,0.666667,0.333333,48.333333,...,10.000000,-411.000000,309.000000,52.000000,54.000000,76.000000,149.000000,0.368794,0.398496,0.558824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,13.466469,240.677088,75.178998,798.095238,96.162183,99.433954,115.113471,0.200000,0.200000,6.666667,...,-12.000000,373.000000,1368.000000,31.000000,59.000000,52.000000,225.000000,0.215278,0.395833,0.346667
5113,6.298669,77.892592,69.699903,860.833333,7.772816,4.078622,7.088127,0.000000,0.000000,5.000000,...,3.000000,2.000000,52.000000,38.000000,56.000000,58.000000,213.000000,0.319149,0.438776,0.387097
5114,-1.250824,173.730990,76.147088,787.948718,110.299038,66.901784,74.622507,0.333333,0.125000,75.000000,...,13.000000,-221.000000,624.000000,44.000000,56.000000,90.000000,182.000000,0.301370,0.384615,0.604027
5115,5.391590,70.402489,71.738000,836.376812,64.642555,65.976396,78.847519,0.523810,0.238095,30.000000,...,4.000000,-23.000000,211.000000,55.000000,91.000000,72.000000,215.000000,0.327381,0.486631,0.384615


In [67]:
model = RandomForestClassifier(n_estimators=1000,max_depth=15)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=15, n_estimators=1000)

In [68]:
model.score(X_val,y_val)

0.7203125

In [69]:
from sklearn.metrics import f1_score
y_pred = model.predict(X_val)
f1 = f1_score(y_val,y_pred,average='weighted')

In [70]:
f1

0.6905569235433318

In [71]:
f1

0.6905569235433318

In [84]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=400,hidden_layer_sizes=[116,232,116,58],verbose=True, n_iter_no_change=1000, learning_rate_init=0.1,learning_rate='adaptive', early_stopping= True)
mlp.fit(X_train,y_train)
pred = mlp.predict(X_val)


Iteration 1, loss = 14.62972491
Validation score: 0.572917
Iteration 2, loss = 4.35660690
Validation score: 0.580729
Iteration 3, loss = 1.02315050
Validation score: 0.580729
Iteration 4, loss = 1.01272385
Validation score: 0.580729
Iteration 5, loss = 1.00790752
Validation score: 0.580729
Iteration 6, loss = 1.00147897
Validation score: 0.580729
Iteration 7, loss = 1.00319255
Validation score: 0.580729
Iteration 8, loss = 1.00166540
Validation score: 0.580729
Iteration 9, loss = 1.00613402
Validation score: 0.580729
Iteration 10, loss = 1.00309816
Validation score: 0.580729
Iteration 11, loss = 1.00597140
Validation score: 0.580729
Iteration 12, loss = 1.00342185
Validation score: 0.580729
Iteration 13, loss = 1.00574257
Validation score: 0.580729
Iteration 14, loss = 1.00364837
Validation score: 0.580729
Iteration 15, loss = 1.00438940
Validation score: 0.580729
Iteration 16, loss = 1.00762005
Validation score: 0.580729
Iteration 17, loss = 1.01069760
Validation score: 0.580729
Itera

Iteration 141, loss = 1.00227942
Validation score: 0.580729
Iteration 142, loss = 1.00661983
Validation score: 0.580729
Iteration 143, loss = 1.00493867
Validation score: 0.580729
Iteration 144, loss = 1.00704600
Validation score: 0.580729
Iteration 145, loss = 1.00436704
Validation score: 0.580729
Iteration 146, loss = 1.00657264
Validation score: 0.580729
Iteration 147, loss = 1.02303132
Validation score: 0.580729
Iteration 148, loss = 1.00440197
Validation score: 0.580729
Iteration 149, loss = 1.00394436
Validation score: 0.580729
Iteration 150, loss = 1.00378131
Validation score: 0.580729
Iteration 151, loss = 1.00154148
Validation score: 0.580729
Iteration 152, loss = 1.00314148
Validation score: 0.580729
Iteration 153, loss = 1.01599754
Validation score: 0.580729
Iteration 154, loss = 1.00605477
Validation score: 0.580729
Iteration 155, loss = 1.00335239
Validation score: 0.580729
Iteration 156, loss = 1.01859434
Validation score: 0.580729
Iteration 157, loss = 1.00272884
Validat

Iteration 280, loss = 1.02130160
Validation score: 0.580729
Iteration 281, loss = 1.00477050
Validation score: 0.580729
Iteration 282, loss = 1.00267008
Validation score: 0.580729
Iteration 283, loss = 1.00587766
Validation score: 0.580729
Iteration 284, loss = 1.00959080
Validation score: 0.580729
Iteration 285, loss = 1.00766588
Validation score: 0.580729
Iteration 286, loss = 1.00278931
Validation score: 0.580729
Iteration 287, loss = 1.00156666
Validation score: 0.580729
Iteration 288, loss = 1.00474028
Validation score: 0.580729
Iteration 289, loss = 1.00111640
Validation score: 0.580729
Iteration 290, loss = 1.00593045
Validation score: 0.580729
Iteration 291, loss = 1.00747446
Validation score: 0.580729
Iteration 292, loss = 1.00614872
Validation score: 0.580729
Iteration 293, loss = 1.00952903
Validation score: 0.580729
Iteration 294, loss = 1.00171066
Validation score: 0.580729
Iteration 295, loss = 1.00252233
Validation score: 0.580729
Iteration 296, loss = 1.00776585
Validat

C:\Users\Samuel\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [85]:
mlp.score(X_val,y_val)

0.603125

In [163]:
from sklearn.ensemble import HistGradientBoostingClassifier

X_train, X_val, y_train, y_val = train_test_split(training_all.iloc[:,:-1], training_all.iloc[:,-1])

hist = HistGradientBoostingClassifier(class_weight='balanced',l2_regularization=0.2, max_depth=7,early_stopping=True)
hist.fit(X_train, y_train)
pred = hist.predict(X_val)

C:\Users\Samuel\anaconda3\Lib\site-packages\pandas\core\generic.py:1998: UserWarning: Warning: converting a masked element to nan.
  arr = np.asarray(values, dtype=dtype)
C:\Users\Samuel\anaconda3\Lib\site-packages\pandas\core\generic.py:1998: UserWarning: Warning: converting a masked element to nan.
  arr = np.asarray(values, dtype=dtype)


In [164]:
f1 = f1_score(y_val,pred,average='weighted')
f1

0.7119750170716498

In [165]:
pred = hist.predict(X_train)
f1 = f1_score(y_train,pred,average='weighted')
f1

C:\Users\Samuel\anaconda3\Lib\site-packages\pandas\core\generic.py:1998: UserWarning: Warning: converting a masked element to nan.
  arr = np.asarray(values, dtype=dtype)


0.9222428833002372

In [166]:
test_data = pd.read_csv('X_test.csv')

In [167]:
test_arrays = []
for row in range(len(test_data)):
    test_arrays.append(test_data.iloc[row,1:].values[~np.isnan(test_data.iloc[row,1:])])

In [171]:
features = []
for X in test_arrays:
        data = truncate(X)
        try:
            n_f = neuro_features(data)
            features.append(np.append(n_f,y))
        except:
            features.append(np.array([np.nan]*33))

C:\Users\Samuel\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Samuel\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Us

In [174]:
hrv_features = []
for X in test_arrays:
        data = truncate(X)
        mean,std = stat_mesaures(data)
        try:
            hp_m = list(hp_measures(data))
            hrv_features.append([mean,std,*hp_m])
        except:
            hrv_features.append([np.nan]*(len(hrv_features[0])))

In [175]:
measures = ['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate', 'vlf', 'lf', 'hf', 'lf/hf', 'p_total', 'vlf_perc', 'lf_perc', 'hf_perc', 'lf_nu', 'hf_nu']
test_data_hrv = pd.DataFrame(hrv_features,columns=['mean','std',*measures])
test_data = pd.DataFrame(features)

In [176]:
test_all = pd.concat([test_data_hrv,test_data],axis = 1)

In [177]:
test_all.replace([np.inf, -np.inf], np.nan, inplace=True)

In [180]:
test_all

,mean,std,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,...,23,24,25,26,27,28,29,30,31,32
0,38.882186,296.223337,70.660522,849.130435,56.219183,22.191669,28.497740,0.272727,0.090909,23.333333,...,307.0,102.0,519.0,60.0,80.0,82.0,203.0,0.339744,0.661157,0.473988
1,26.169618,158.642401,54.802104,1094.848485,52.384816,48.379288,63.848780,0.400000,0.200000,23.333333,...,4.0,-101.0,125.0,58.0,162.0,44.0,257.0,0.277512,0.801980,0.211538
2,5.247077,260.639815,154.772141,387.666667,117.265605,59.207732,205.385816,1.000000,1.000000,90.000000,...,205.0,498.0,820.0,82.0,98.0,55.0,207.0,0.546667,0.547486,0.327160
3,14.085320,94.224880,62.500000,960.000000,15.731330,9.991178,17.563189,0.285714,0.000000,10.000000,...,104.0,-17.0,218.0,47.0,78.0,61.0,234.0,0.288344,0.458333,0.371951
4,28.742796,164.445141,60.713138,988.253968,55.335619,36.695443,67.230945,0.700000,0.550000,30.000000,...,6.0,-80.0,67.0,49.0,77.0,38.0,204.0,0.278409,0.509934,0.212291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3406,-12.033701,161.658572,89.361702,671.428571,120.473367,159.729079,179.722008,0.285714,0.142857,18.333333,...,-20.0,117.0,364.0,43.0,57.0,63.0,174.0,0.270440,0.422222,0.406452
3407,1.922526,179.142622,62.780269,955.714286,33.080675,30.09245,67.123435,0.750000,0.750000,23.333333,...,102.0,923.0,1111.0,47.0,50.0,51.0,225.0,0.260870,0.273292,0.314815
3408,13.263693,127.432297,79.009858,759.398907,56.211636,26.766189,46.498891,0.677966,0.338983,43.333333,...,65.0,-95.0,263.0,54.0,88.0,59.0,189.0,0.355263,0.527950,0.450382
3409,12.638384,92.841658,81.985236,731.839080,7.710579,3.632416,5.527708,0.000000,0.000000,3.333333,...,26.0,-10.0,144.0,46.0,50.0,48.0,170.0,0.282209,0.323741,0.320000


In [186]:
X_train = training_all.iloc[:,:-1]
y_train = training_all.iloc[:,-1]
test_all.columns = test_all.columns.astype(str)
hist = HistGradientBoostingClassifier(class_weight='balanced',l2_regularization=0.2, max_depth=7)
hist.fit(X_train, y_train)
pred = hist.predict(test_all)

C:\Users\Samuel\anaconda3\Lib\site-packages\pandas\core\generic.py:1998: UserWarning: Warning: converting a masked element to nan.
  arr = np.asarray(values, dtype=dtype)
C:\Users\Samuel\anaconda3\Lib\site-packages\pandas\core\generic.py:1998: UserWarning: Warning: converting a masked element to nan.
  arr = np.asarray(values, dtype=dtype)


In [188]:
y_test = pd.DataFrame(pred,columns = ['y']).astype('int32').to_csv('sol.csv',index_label = 'id')

In [189]:
test_all.isna().sum().sum()

13300